<a href="https://colab.research.google.com/github/jgmsgabriel/formacao_LLM/blob/main/Joao_Curso_LLMs_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers einops accelerate bitsandbytes
!pip install -q langchain langchain_community langchain-huggingface langchainhub langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.1 MB/s eta 0:00:

In [ ]:
import torch
import os
import getpass

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
os.environ['HF_TOKEN'] = getpass.getpass()

··········


In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
)

llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
# LLAMA 3
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Você é um assistente virtual prestativo e está respondendo perguntas gerais.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{pergunta}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

template

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{pergunta}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

In [ ]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{pergunta}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [ ]:
chain = prompt | llm

chain.invoke({"pergunta": "Que dia é hoje?"})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Peço desculpas, mas como sou um assistente virtual, não tenho acesso a informações em tempo real sobre a data atual. No entanto, posso ajudá-lo com outras coisas!'

## Prompt com RAG

In [ ]:
template_rag = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Você é um assistente virtual prestativo e está respondendo perguntas gerais.
Use os seguintes pedaços de contexto recuperado para responder à pergunta.
Se você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Pergunta: {pergunta}
Contexto: {contexto}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [ ]:
prompt_rag = PromptTemplate.from_template(template_rag)
prompt_rag

PromptTemplate(input_variables=['contexto', 'pergunta'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [ ]:
from datetime import date

dia = date.today()
print(dia)

2025-01-23


In [ ]:
contexto = "Você sabe que hoje é dia '{}'".format(dia)
print(contexto)

Você sabe que hoje é dia '2025-01-23'


In [ ]:
chain_rag = prompt_rag | llm | StrOutputParser()

pergunta = "Qual dia é hoje? Responta utilizando o formate dd/mm/yyyy"

res = chain_rag.invoke({"pergunta": pergunta, "contexto": contexto})
res

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Hoje é 23/01/2025.'

In [ ]:
prompt_rag

PromptTemplate(input_variables=['contexto', 'pergunta'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [ ]:
chain_rag = prompt_rag | llm | StrOutputParser()

contexto = """Faturamento trimestral:
1º: R$42476,40
2º: R$46212,97
3º: R$101324,56
4º: R$56430,24"""

#pergunta = "Qual é o faturamento do segundo trimestre?"
pergunta = "Qual trimestre teve o maior faturamento?"

chain_rag.invoke({
  "contexto": contexto,
  "pergunta": pergunta
})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'O trimestre com o maior faturamento foi o 3º, com R$101324,56.'

## Depuração/Debugging

In [ ]:
from langchain.globals import set_debug
set_debug(True)

In [ ]:
pergunta = "Qual trimestre teve o menor faturamento?"

chain_rag.invoke({
  "contexto": contexto,
  "pergunta": pergunta
})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "contexto": "Faturamento trimestral:\n1º: R$42476,40\n2º: R$46212,97\n3º: R$101324,56\n4º: R$56430,24",
  "pergunta": "Qual trimestre teve o menor faturamento?"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "contexto": "Faturamento trimestral:\n1º: R$42476,40\n2º: R$46212,97\n3º: R$101324,56\n4º: R$56430,24",
  "pergunta": "Qual trimestre teve o menor faturamento?"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha 

'O trimestre com o menor faturamento foi o 1º, com R$42.476,40.'

In [ ]:
set_debug(True)

## Aplicação para RAG com contextos maiores

### 01 - Carregando o conteúdo

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [ ]:
loader = WebBaseLoader(web_paths = ("https://www.bbc.com/portuguese/articles/cd19vexw0y1o",),)
docs = loader.load()

In [ ]:
len(docs[0].page_content)

11897

In [ ]:
docs

[Document(metadata={'source': 'https://www.bbc.com/portuguese/articles/cd19vexw0y1o', 'title': 'Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News Brasil', 'description': "'Oppenheimer' foi o grande vencedor da noite com sete estatuetas, incluindo o prêmio de melhor filme, melhor diretor e melhor ator.", 'language': 'pt-br'}, page_content="Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News BrasilBBC News, BrasilVá para o conteúdoSeçõesNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosBBC LêNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosBBC LêOscar 2024: confira todos os ganhadores dos prêmios da Academia de HollywoodCrédito, Getty ImagesLegenda da foto, Robert Downey Jr., Da'Vine Hoy Randolph, Emma Stone e Cillian Murphy com suas respectivas estatuetas do OscarArticle informationAuthor, Leire VentasRole,  Correspondente da BBC News Mundo em Los AngelesTwitter, @leire_ventas11 março 

In [ ]:
print(docs[0].page_content[:300])

Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News BrasilBBC News, BrasilVá para o conteúdoSeçõesNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosBBC LêNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosBBC LêOscar 2024: confira todos o


### 02 - Divisão em pedaços de texto / Split


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, add_start_index = True)
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

15

In [ ]:
splits[3]

Document(metadata={'source': 'https://www.bbc.com/portuguese/articles/cd19vexw0y1o', 'title': 'Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News Brasil', 'description': "'Oppenheimer' foi o grande vencedor da noite com sete estatuetas, incluindo o prêmio de melhor filme, melhor diretor e melhor ator.", 'language': 'pt-br', 'start_index': 2405}, page_content='Nolan – OppenheimerMelhor atriz: Emma Stone – Pobres CriaturasMelhor ator: Cillian Murphy - OppenheimerMelhor atriz coadjuvante: Da’Vine Joy Randolph - Os RejeitadosMelhor ator coadjuvante: Robert Downey Jr. - OppenheimerMelhor roteiro original: Anatomia de Uma Queda— Justin Triet, Arthur HarariMelhor roteiro adaptado: American Fiction - Cord JeffersonMelhor edição: OppenheimerMelhor filme estrangeiro: Zona de InteresseMelhor animação: O Menino e a GarçaMelhor curta-metragem de animação: War Is Over! Inspired by the Music of John & Yoko - Dace Mullins, Brad BookerMelhor maquiagem e penteados: 

In [ ]:
splits[3].metadata

{'source': 'https://www.bbc.com/portuguese/articles/cd19vexw0y1o',
 'title': 'Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News Brasil',
 'description': "'Oppenheimer' foi o grande vencedor da noite com sete estatuetas, incluindo o prêmio de melhor filme, melhor diretor e melhor ator.",
 'language': 'pt-br',
 'start_index': 2405}

### 3 - Armazenamento


In [ ]:
hf_embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

In [ ]:
input_test = "Um teste apenas"
result = hf_embeddings.embed_query(input_test)

In [ ]:
len(result)

768

In [ ]:
print(result)

[-0.026895053684711456, -0.011064697988331318, -0.04531998932361603, -0.0013973648892715573, 0.042436011135578156, -0.014201739802956581, 0.023354941979050636, 0.06011558324098587, 0.061521951109170914, 0.006502094678580761, 0.008159150369465351, -0.030567463487386703, 0.002060323255136609, 0.01296155247837305, 0.004251229576766491, 0.0036631699185818434, -0.026755528524518013, 0.029737044125795364, -0.009770004078745842, -0.046504564583301544, -0.028108958154916763, 0.00016858700837474316, -0.024811016395688057, -0.011828277260065079, 0.08182830363512039, 0.0014993291115388274, 0.013264901004731655, -0.06242178753018379, -0.0012287781573832035, 0.02897716872394085, -0.029528986662626266, -0.025696074590086937, 0.003377106273546815, -0.028868991881608963, 1.518518047305406e-06, -0.03882475197315216, -0.019842026755213737, -0.01765800639986992, -0.008112344890832901, -0.02513054944574833, 0.025749174878001213, 0.11367511004209518, -0.006133120507001877, -0.017986472696065903, -0.0454115

In [ ]:
vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)

### 4 - Configurando o recuperador de texto / Retriever


In [ ]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": 6})

### 5 - Geração

In [ ]:
template_rag

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

In [ ]:
prompt_rag = PromptTemplate(
    input_variables=["contexto", "pergunta"],
    template=template_rag,
)
prompt_rag

PromptTemplate(input_variables=['contexto', 'pergunta'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [ ]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
chain_rag = ({"contexto": retriever | format_docs, "pergunta": RunnablePassthrough()}
             | prompt_rag
             | llm
             | StrOutputParser())

In [ ]:
chain_rag.invoke("Qual filme ganhou mais oscars na premiação de 2024?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Qual filme ganhou mais oscars na premiação de 2024?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<contexto,pergunta>] Entering Chain run with input:
{
  "input": "Qual filme ganhou mais oscars na premiação de 2024?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<contexto,pergunta> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Qual filme ganhou mais oscars na premiação de 2024?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<contexto,pergunta> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Qual filme ganhou mais oscars na premiação de 2024?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<contexto,pergunta> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Qual filme ganhou mais oscars na premiação de 2024?"
}
[chain/start] [chain:RunnableSequence > chain:

'Infelizmente, não há informações disponíveis sobre o filme que ganhou mais Oscars na premiação de 2024, pois a informação fornecida refere-se ao Oscar de 2025. Além disso, a lista de vencedores do Oscar de 2024 não está presente na matéria fornecida. Se você tiver acesso a essa informação, sinta-se à vontade para compartilhá-la comigo.'

In [ ]:
chain_rag.invoke("Quem ganhou o premio de melhor ator?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Quem ganhou o premio de melhor ator?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<contexto,pergunta>] Entering Chain run with input:
{
  "input": "Quem ganhou o premio de melhor ator?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<contexto,pergunta> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Quem ganhou o premio de melhor ator?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<contexto,pergunta> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Quem ganhou o premio de melhor ator?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<contexto,pergunta> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Quem ganhou o premio de melhor ator?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<contexto,pergunta> > chain:RunnableSequence > chain:format

'A pergunta é: Quem ganhou o prêmio de melhor ator?\n\nA resposta é: Não há menção específica sobre quem ganhou o prêmio de melhor ator nessa reportagem. No entanto, é mencionado que Cillian Murphy ganhou o Oscar de melhor ator por interpretar J. Robert Oppenheimer, o cientista-chefe do Projeto Manhattan.'

In [ ]:
vectorstore.delete_collection()